In [2]:
import os
import numpy as np
import tarfile
from pathlib import Path

import polars as pl
from huggingface_hub import snapshot_download
from polars import selectors as cs
from sklearn.model_selection import train_test_split


/Users/arix/Documents/skoltech/seqmod/embedding_alignment/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
os.environ["DATA_DIR"] = "./data"

In [5]:
raw_path = Path(os.environ["DATA_DIR"], "raw", "mbd_mini")
raw_path.mkdir(exist_ok=True, parents=True)
snapshot_download(
    repo_id="ai-lab/MBD-mini",
    repo_type="dataset",
    local_dir=raw_path,
)

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 9982.48it/s]


'/Users/arix/Documents/skoltech/seqmod/embedding_alignment/data/raw/mbd_mini'

In [6]:
if not (raw_path / "detail").exists():
    with tarfile.open(raw_path / "detail.tar.gz", "r:gz") as tar:
        tar.extractall(raw_path)

In [7]:
if not (raw_path / "targets").exists():
    with tarfile.open(raw_path / "targets.tar.gz", "r:gz") as tar:
        tar.extractall(raw_path)

In [8]:
dialog = pl.read_parquet(raw_path / "detail" / "dialog").drop_nulls()
print(dialog)

shape: (217_205, 4)
┌─────────────────────────────────┬────────────────────────────┬─────────────────────────┬──────┐
│ client_id                       ┆ event_time                 ┆ embedding               ┆ fold │
│ ---                             ┆ ---                        ┆ ---                     ┆ ---  │
│ str                             ┆ datetime[ns]               ┆ list[f32]               ┆ i64  │
╞═════════════════════════════════╪════════════════════════════╪═════════════════════════╪══════╡
│ 5226901c78c655767ac2a075e6edca… ┆ 2021-06-18 05:49:23.962741 ┆ [0.393141, -0.036111, … ┆ 0    │
│                                 ┆                            ┆ 0.3971…                 ┆      │
│ 5226901c78c655767ac2a075e6edca… ┆ 2022-06-29 07:39:31.618769 ┆ [0.302206, -0.381503, … ┆ 0    │
│                                 ┆                            ┆ 0.2886…                 ┆      │
│ 5226901c78c655767ac2a075e6edca… ┆ 2022-06-24 06:06:59.691367 ┆ [0.570911, -0.395088, … ┆ 0    │


In [11]:
emb_example = np.array(dialog["embedding"].item(0))
emb_example.shape, emb_example.min(), emb_example.max()

((768,), np.float32(-0.99998313), np.float32(0.99996156))